# Reduce GIWAXS Transformation

In [ ]:
from pathlib import Path
import pyFAI
import fabio
import numpy as np
import yaml
import matplotlib.pylab as plt
from matplotlib.colors import LogNorm
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

FIG_SIZE = (7.5, 3.75)

raw_stitch = fabio.open("image-transformed.tif").data.astype(np.float64)
flat_field = fabio.open("flat-field-transformed.tif").data.astype(np.float64)
with open("params.yaml", "r") as yf:
    params = yaml.safe_load(yf)
    exposure_time = params["exposure"]
    incident_angle = params["incident"]
    try:
        tilt_angle = params["tilt"]
    except KeyError:
        tilt_angle = 0
if tilt_angle is None:
    titl_angle = 0
print(f"Exposure time  = {exposure_time} s / (2 images)")
print(f"incident angle = {incident_angle} degrees")
print(f"tilt angle     = {tilt_angle} degrees")

print('Generated "fake" flat field from exposure time at each pixel')

fig = plt.figure(figsize=FIG_SIZE, facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(raw_stitch, norm=LogNorm(1, np.max(raw_stitch)))
ax1.set_title("Transformed image")
ax1.set_xlabel("column (pixels)")
ax1.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax1, shrink=0.7)

fig = plt.figure(figsize=FIG_SIZE, facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(flat_field * exposure_time)
ax1.set_title("Exposure time per pixel (s)")
ax1.set_xlabel("column (pixels)")
ax1.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax1, shrink=0.7)

## Adust pixel weights for presentation of stitch

In [ ]:
adjusted_stitch = raw_stitch / flat_field
adjusted_stitch[np.where(adjusted_stitch == np.infty)] = 0
adjusted_stitch = np.nan_to_num(adjusted_stitch)
print(adjusted_stitch.max())

fig = plt.figure(figsize=FIG_SIZE, facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(adjusted_stitch, norm=LogNorm(1, np.max(adjusted_stitch)))
ax1.set_title("Transformed image with exposure times adusted")
ax1.set_xlabel("column (pixels)")
ax1.set_ylabel("row (pixels)")
fig.colorbar(pos, ax=ax1, shrink=0.7)
plt.savefig("stitch.png", dpi=1000)

## Create Cake and reduce data
You must have a `cal-transformed.poni` file. This should have automatically been generated

You can create a mask using pyFAI-calib2 GUI

In [ ]:
ai = pyFAI.load("cal-transformed.poni")
print("Loaded geometry:")
print(ai)
print("")

try:
    mask = fabio.open("mask.edf").data.astype(bool)
    print("Used mask")
except FileNotFoundError:
    mask = np.zeros(raw_stitch.shape)
    print("Did not load mask, you can create one using pyFAI-calib2")

"""Mask pixels that have 0 in the flat field"""
mask = np.logical_or(mask, np.logical_not(flat_field))

#### Create Cake

In [ ]:
q_bins = 500
azimuthal_bins = 180
cake = ai.integrate2d_ng(
    raw_stitch, q_bins, azimuthal_bins,
    radial_range=None,          # In units specified below
    azimuth_range=(180, 360),   # this is 0 to 180 from the right going counter-clockwise
    mask=mask, flat=flat_field,
    error_model="poisson",  unit="q_A^-1",
    polarization_factor=None, correctSolidAngle=False,
)

In [ ]:
fig = plt.figure(figsize=(5, 3), facecolor="w")
ax1 = plt.subplot()
pos = ax1.imshow(cake[0], norm=LogNorm(1, np.max(cake[0])),
                 extent=(np.min(cake[1]), np.max(cake[1]), 0, 180),
                 aspect='auto')
ax1.set_title("Cake")
ax1.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax1.set_ylabel(r"$\psi$ (degrees)")
ax1.set_yticks(np.arange(0, 181, 20))
ax1.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
# ax1.set_xticks(np.arange(0, np.min(cake[1]) + np.max(cake[1]), 0.5))
ax1.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
fig.colorbar(pos, ax=ax1)
plt.savefig("cake.png", dpi=1000, bbox_inches="tight")

#### Create Reduction

In [ ]:
q_bins = 1000
file_to_save = "reduction.edf"
if (Path.cwd() / file_to_save).is_file():
    (Path.cwd() / file_to_save).unlink()

redu = ai.integrate1d_ng(
    raw_stitch, q_bins,
    radial_range=None,          # In units specified below
    azimuth_range=(180, 360),   # this is 0 to 180 from the right going counter-clockwise
    mask=mask, flat=flat_field, error_model="poisson",
    unit="q_A^-1", filename="reduction.edf", normalization_factor=(exposure_time / 60)
)

In [ ]:
plt.figure(figsize=(4, 3))
ax2 = plt.subplot()

ax2.scatter(
    redu[0], redu[1],
    s=5,  # marker size
    marker="o",  # marker shape
    edgecolors="black",  # marker edge color
    lw=.75,  # marker edge width
    alpha=1,  # transparency
    facecolor='w'  # marker face color
)

ax2.set_title("Reduction")
ax2.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
# ax2.set_xticks(np.arange(0, np.min(redu[0]) + np.max(redu[0]), 0.5))
ax2.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
ax2.set_xlabel(r"$q\ (\mathregular{\AA}^{-1})$")
ax2.grid()
ax2.set_ylabel(r"Intensity rate (counts / (min $\cdot$ projected pixel area)")
ax2.set_yscale("log")
plt.savefig("reduction.png", dpi=1000, bbox_inches="tight")